In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 1, 0])              # Boundary condition function coefficients
dtype = torch.float                  # Tensor data type
supervised_samples = {n for n in range(128)}
unsupervised_samples = {n for n in range(0)}

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
#Specify right hand side and stiffness matrix
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
K = PoissonFEM.StiffnessMatrix(mesh)
rhs.set_rhs_stencil(mesh, K)

In [5]:
trainingData = dta.StokesData(supervised_samples, unsupervised_samples)
trainingData.read_data()
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs, trainingData.output_resolution**2)

In [7]:
model = gs.GenerativeSurrogate(rom, trainingData)
# model = gs.GenerativeSurrogate()
# model.load()

In [8]:
model.fit(n_steps=10, with_precisions=False)

step =  0
loss z =  5868882.5
loss z =  5868862.0
loss z =  5868846.5
loss z =  5868829.5
loss z =  5868814.5
loss z =  5868797.5
loss z =  5868780.0
loss z =  5868764.5
loss z =  5868748.0
z step =  11.842623949050903 s
loss_f =  5673285.0
loss_f =  5571768.5
loss_f =  5488559.0
loss_f =  5415919.5
loss_f =  5350365.5
loss_f =  5290017.5
loss_f =  5233717.5
loss_f =  5180741.0
loss_f =  5130581.5
thetaf step =  15.98843002319336 s
sample ==  0
log_lambdac_pred =  tensor([[-0.1929,  0.0370,  0.1348,  ...,  0.1127, -0.1735, -0.1112],
        [-0.1978,  0.0325,  0.1394,  ...,  0.1139, -0.1784, -0.1038],
        [-0.1922,  0.0323,  0.1468,  ...,  0.1200, -0.1705, -0.1117],
        ...,
        [-0.1906,  0.0351,  0.1380,  ...,  0.1206, -0.1790, -0.1129],
        [-0.1965,  0.0421,  0.1479,  ...,  0.1186, -0.1771, -0.1133],
        [-0.1994,  0.0353,  0.1410,  ...,  0.1152, -0.1738, -0.1096]],
       grad_fn=<AddmmBackward>)
log_lambdac_pred =  tensor([[-0.1929,  0.0370,  0.1348,  ...,  0.

AssertionError: 

In [ ]:
model.fit(n_steps=50, with_precisions=True)

In [10]:
model.log_lambdac_mean

tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [-7., -7., -7.,  ..., -7., -7., -7.],
        [-7., -7., -7.,  ..., -7., -7., -7.],
        ...,
        [-7., -7., -7.,  ..., -7., -7., -7.],
        [-7., -7., -7.,  ..., -7., -7., -7.],
        [-7., -7., -7.,  ..., -7., -7., -7.]], requires_grad=True)

In [ ]:
model = gs.GenerativeSurrogate()
model.load()

In [ ]:
test_samples = {n for n in range(0, 8)}
testData = dta.StokesData(unsupervised_samples=test_samples)
testData.read_data()
# trainingData.plotMicrostruct(1)
testData.reshape_microstructure_image()

In [ ]:
uf_pred, Z = model.predict(testData)

In [ ]:
uf_pred[4]

In [ ]:
model.fit(n_steps=10)

In [ ]:
x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose

In [ ]:
fig = plt.figure()

In [ ]:
ax = plt.axes(projection='3d')

In [ ]:
ax.plot_surface(x, y, np.reshape(model.rom.interpolated_solution.array, (129, 129)),cmap='viridis', edgecolor='none')

In [ ]:
pp = np.reshape(model.data.P[15, :], (129, 129))

In [ ]:
ax.plot_surface(x, y, pp.detach().numpy())

In [ ]:
model.rom.full_solution.array

In [ ]:
model.log_lambdac_mean

In [ ]:
model.log_lambda_c_mean

In [ ]:
True % 10

In [ ]:
rom.solution.array

In [ ]:
y = torch.tensor([1, 2, 3])

In [ ]:
y  + 1.2

In [ ]:
model.tauc